<a href="https://colab.research.google.com/github/asifshigri12/notebooks/blob/main/llama_pack_resume_July_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Llama Pack - Resume Screener 📄


You can find all packs on https://llamahub.ai

The resume screener is designed to analyze a candidate's resume according to a set of criteria, and decide whether the candidate is a fit for the job.

in this example we'll evaluate a sample resume.

In [ ]:
%pip install llama-index-readers-wikipedia

In [ ]:
!pip install llama-index llama-hub wikipedia

### Setup Data

We'll load some sample Wikipedia data for OpenAI, Sam, Mira, and Emmett. Why? No reason in particular :)

In [ ]:
from llama_index.readers.wikipedia import WikipediaReader

loader = WikipediaReader()
documents = loader.load_data(
    pages=["OpenAI", "Sam Altman", "Mira Murati", "Emmett Shear"],
    auto_suggest=False,
)

In [ ]:
# do sentence splitting on the first piece of text
from llama_index.core.node_parser import SentenceSplitter

In [ ]:
sentence_splitter = SentenceSplitter(chunk_size=1024)

We get the first chunk from each essay.

In [ ]:
# get the first 1024 tokens for each entity
openai_node = sentence_splitter.get_nodes_from_documents([documents[0]])[0]
sama_node = sentence_splitter.get_nodes_from_documents([documents[1]])[0]
mira_node = sentence_splitter.get_nodes_from_documents([documents[2]])[0]
emmett_node = sentence_splitter.get_nodes_from_documents([documents[3]])[0]

We'll also download Jerry's resume in 2019.

## Download Resume Screener Pack from LlamaHub

Here we download the resume screener pack class from LlamaHub.

We'll use it for two use cases:
- whether the candidate is a good fit for a front-end / full-stack engineering role.
- whether the candidate is a good fit for the CEO of OpenAI.

In [ ]:
from llama_index.core.llama_pack import download_llama_pack

In [ ]:
ResumeScreenerPack = download_llama_pack(
    "ResumeScreenerPack", "./resume_screener_pack"
)

### Screen Candidate for MLE Role

We take a job description on an MLE role from Meta's website.

In [ ]:
meta_jd = """\
Meta is embarking on the most transformative change to its business and technology in company history, and our Machine Learning Engineers are at the forefront of this evolution. By leading crucial projects and initiatives that have never been done before, you have an opportunity to help us advance the way people connect around the world.

The ideal candidate will have industry experience working on a range of recommendation, classification, and optimization problems. You will bring the ability to own the whole ML life cycle, define projects and drive excellence across teams. You will work alongside the world’s leading engineers and researchers to solve some of the most exciting and massive social data and prediction problems that exist on the web.\
"""

In [ ]:
import os
from google.colab import userdata
#os.environ['OPENAI_API_KEY'] = userdata.get('openAI2024')
#You need to run below cell
os.environ['OPENAI_API_KEY'] = " Add API Key Here "
# # Replace 'YOUR_API_KEY' with your actual OpenAI API key


# os.environ['OPENAI_API_KEY'] = ''

In [ ]:
resume_screener = ResumeScreenerPack(
    job_description=meta_jd,
    criteria=[
        "2+ years of experience in one or more of the following areas: machine learning, recommendation systems, pattern recognition, data mining, artificial intelligence, or related technical field",
        "Experience demonstrating technical leadership working with teams, owning projects, defining and setting technical direction for projects",
        "Bachelor's degree in Computer Science, Computer Engineering, relevant technical field, or equivalent practical experience.",
        "Must be from USA",
    ],
)

In [ ]:
response = resume_screener.run(resume_path="/content/cv_Umair_ML_25.pdf")

In [ ]:
for cd in response.criteria_decisions:
    print("### CRITERIA DECISION")
    print(cd.reasoning)
    print(cd.decision)
print("#### OVERALL REASONING ##### ")
print(str(response.overall_reasoning))
print(str(response.overall_decision))

### CRITERIA DECISION
The candidate has extensive experience in machine learning, recommendation systems, pattern recognition, data mining, and artificial intelligence. He has led over 45 AI, ML, DL, and Generative AI projects, demonstrating technical leadership.
True
### CRITERIA DECISION
The candidate has demonstrated technical leadership in his roles as an AI Trainer at AtomCamp and as a consultant for various companies. He has also defined and set the technical direction for projects, as evidenced by his work on various AI and ML projects.
True
### CRITERIA DECISION
The candidate has a PhD in a relevant technical field from the University of Electronic Science and Technology China, and an MS in Computer Science from National Textile University, Faisalabad.
True
### CRITERIA DECISION
The candidate is not from the USA. His address is listed as Lahore, Pakistan.
False
#### OVERALL REASONING ##### 
While the candidate meets most of the criteria, including having relevant experience and

### Screen Candidate for FE / Typescript roles

In [ ]:
resume_screener = ResumeScreenerPack(
    job_description="We're looking to hire a front-end engineer",
    criteria=[
        "The individual needs to be experienced in front-end / React / Typescript"
    ],
)

In [ ]:
response = resume_screener.run(resume_path="/content/cv_Umair_ML_18.pdf")

In [ ]:
print(str(response.overall_reasoning))
print(str(response.overall_decision))

The candidate does not meet the specific criteria for the front-end engineer position as there is no evidence of experience or skills in front-end development, React, or Typescript.
False


### Screen Candidate for CEO of OpenAI

Jerry can't write Typescript, but can he be CEO of OpenAI?

In [ ]:
job_description = f"""\
We're looking to hire a CEO for OpenAI.

Instead of listing a set of specific criteria, each "criteria" is instead a short biography of a previous CEO.\

For each criteria/bio, outline if the candidate's experience matches or surpasses that of the candidate.

Also, here's a description of OpenAI from Wikipedia:
{openai_node.get_content()}
"""

profile_strs = [
    f"Profile: {n.get_content()}" for n in [sama_node, mira_node, emmett_node]
]


resume_screener = ResumeScreenerPack(
    job_description=job_description, criteria=profile_strs
)

In [ ]:
response = resume_screener.run(resume_path="jerry_resume.pdf")

In [ ]:
for cd in response.criteria_decisions:
    print("### CRITERIA DECISION")
    print(cd.reasoning)
    print(cd.decision)
print("#### OVERALL REASONING ##### ")
print(str(response.overall_reasoning))
print(str(response.overall_decision))

...sadly not

# Gradio App

In [ ]:
!pip install gradio

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.2/318.2 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 60.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.4/341.4 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 66.6 MB/s eta 0:00:00
   ━━━

In [ ]:
import gradio as gr
import os
from llama_index.readers.wikipedia import WikipediaReader
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.llama_pack import download_llama_pack
from google.colab import userdata

# Load Wikipedia pages
loader = WikipediaReader()
documents = loader.load_data(
    pages=["OpenAI", "Sam Altman", "Mira Murati", "Emmett Shear"],
    auto_suggest=False,
)

# Split documents into nodes
sentence_splitter = SentenceSplitter(chunk_size=1024)

openai_node = sentence_splitter.get_nodes_from_documents([documents[0]])[0]
sama_node = sentence_splitter.get_nodes_from_documents([documents[1]])[0]
mira_node = sentence_splitter.get_nodes_from_documents([documents[2]])[0]
emmett_node = sentence_splitter.get_nodes_from_documents([documents[3]])[0]

# Download Llama pack
ResumeScreenerPack = download_llama_pack("ResumeScreenerPack", "./resume_screener_pack")

# Job description and criteria
meta_jd = """\
Meta is embarking on the most transformative change to its business and technology in company history, and our Machine Learning Engineers are at the forefront of this evolution. By leading crucial projects and initiatives that have never been done before, you have an opportunity to help us advance the way people connect around the world.

The ideal candidate will have industry experience working on a range of recommendation, classification, and optimization problems. You will bring the ability to own the whole ML life cycle, define projects and drive excellence across teams. You will work alongside the world’s leading engineers and researchers to solve some of the most exciting and massive social data and prediction problems that exist on the web.\
"""

criteria = [
    "2+ years of experience in one or more of the following areas: machine learning, recommendation systems, pattern recognition, data mining, artificial intelligence, or related technical field",
    "Experience demonstrating technical leadership working with teams, owning projects, defining and setting technical direction for projects",
    "Bachelor's degree in Computer Science, Computer Engineering, relevant technical field, or equivalent practical experience.",
]

# Set OpenAI API key
os.environ['OPENAI_API_KEY'] = userdata.get('openAI2024')

# Define the resume screening function
def screen_resume(resume):
    resume_screener = ResumeScreenerPack(
        job_description=meta_jd,
        criteria=criteria,
    )
    response = resume_screener.run(resume_path=resume)

    decisions = []
    for cd in response.criteria_decisions:
        decisions.append(f"### CRITERIA DECISION\n{cd.reasoning}\nDecision: {cd.decision}\n")

    overall_reasoning = f"#### OVERALL REASONING #####\n{str(response.overall_reasoning)}\nOverall Decision: {str(response.overall_decision)}"

    return "\n".join(decisions) + "\n" + overall_reasoning

# Create the Gradio interface
iface = gr.Interface(
    fn=screen_resume,
    inputs=gr.File(file_types=[".pdf"]),
    outputs="text",
    title="Resume Screener",
    description="Upload a PDF resume to screen it according to the provided job description and criteria."
)

# Launch the interface
iface.launch(auth=("atomcamp", "12345678"))


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://e63151a9ae7e7546c9.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
